In [ ]:
import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import  gc
from pyxirr import xirr
from tqdm.notebook import tqdm

from tqdm.notebook import tqdm
trigger = [100,25,50,75,125,150, 175, 200]
delta_list=['50','45', '40', '35', '30', '25','20','15','10','5']
list1=[100,25,50,125,150,175,200]
N1=[3]#,1,2]#3,1,2]#,3,4,0]
import warnings
warnings.filterwarnings("ignore")
leverage=10
def closest_strike(group, input_value,opt_type):
    a=(np.abs(group['OPT_Close'] - input_value)).argmin()

    atm=group.iloc[0]['At_The_Money']
    
    if opt_type=='CE':
        if group.iloc[a]['Strike']<group.iloc[a]['At_The_Money']:
            # print(111111,a)

            a=group.index[group['Strike'] == atm][0]
    if opt_type=='PE':
        
        if group.iloc[a]['Strike']>group.iloc[a]['At_The_Money']:
            a=group.index[group['Strike'] == atm][0]
    
    return int(a)
    
    
def adjust(group):
    global idx
    # print(112)
    temp=0
    start_date=group["Date"].iat[0]
    end_date=group["Date"].iat[-1]
    idx_new=idx[(idx['Date']<=pd.to_datetime(end_date)) & (idx['Date']>=pd.to_datetime(start_date))].reset_index(drop=True)
    # display(idx_new)
    # print(group)
    temp1=0
    temp2=0
    # display(group)
    for idi,row in group.iterrows():
        # print(idi)
        if row['Status1']==1:
            temp2=1
            continue
            # print(shift_date)
            # if group.loc[idi-1,'Marker3']=='F':
            #     break
        if temp2==1:
            shiftdate=row['Date']
            temp2=0
            # display(row)
            # print(1,shiftdate)
            group.loc[idi,'Marker3']='F'
            group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']
            group.loc[idi, 'stop_trigger_CE'] = t*(group.loc[idi, 'Strike_CE'] + group.loc[idi, 'Premium'])/100
            group.loc[idi, 'stop_trigger_PE'] = t*(group.loc[idi,'Strike_PE'] - group.loc[idi, 'Premium'])/100
            group.loc[idi+1:,'stop_trigger_CE'] = np.nan
            group.loc[idi+1:,'stop_trigger_PE'] = np.nan
            group['stop_trigger_CE'].ffill(inplace=True)
            group['stop_trigger_PE'].ffill(inplace=True)
            temp=1
            # idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "CE")]
            ce_strike=idx_new[(idx_new['Date']==shiftdate) &(idx_new["Strike"] == idx_new["Delta_"+delta+"_Strike"]) & (idx_new["Option_Type"] == "CE")]['Strike'].iat[0]
            pe_strike=idx_new[(idx_new['Date']==shiftdate)& (idx_new["Strike"] == idx_new["Delta_"+delta+"_Strike"]) & (idx_new["Option_Type"] == "PE")]['Strike'].iat[0]
            # print(ce_strike,pe_strike)
        if temp==1:
            if pd.to_datetime(row['Date'])>=pd.to_datetime(shiftdate):
                # print
                if row['Marker']!='L' and temp1==0:
                    temp1=1
                if temp1==1:
                    curr_date=row['Date']
                    x_ce=idx_new[(idx_new['Date']==curr_date) & (idx_new["Option_Type"] == "CE") & (idx_new["Strike"]==ce_strike)]
                    x_pe=idx_new[(idx_new['Date']==curr_date) & (idx_new["Option_Type"] == "PE") & (idx_new["Strike"]==pe_strike)]
                    # display(x_ce)
                    # display(x_pe)
                    if len(x_ce):
                        group.loc[idi,'Strike_CE']=x1=x_ce["Strike"].iat[0]
                        group.loc[idi,'OPT_Close_CE']=x2=x_ce["OPT_Close"].iat[0]
                        group.loc[idi,'Ticker_CE']=x3=x_ce["Ticker"].iat[0]
                        group.loc[idi,'IV_CE']=x_ce["IV"].iat[0]
                    if len(x_ce)==0:
                        group.loc[idi,'Strike_CE']=x1#x["Strike"].iat[0]
                        group.loc[idi,'OPT_Close_CE']=x2#x["OPT_Close"].iat[0]
                        group.loc[idi,'Ticker_CE']=x3#x["Ticker"].iat[0]
                        group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]

                    if len(x_pe):
                        group.loc[idi,'Strike_PE']=y1=x_pe["Strike"].iat[0]
                        group.loc[idi,'OPT_Close_PE']=y2=x_pe["OPT_Close"].iat[0]
                        group.loc[idi,'Ticker_PE']=y3=x_pe["Ticker"].iat[0]
                        group.loc[idi,'IV_PE']=x_pe["IV"].iat[0]
                    if len(x_pe)==0:
                        group.loc[idi,'Strike_PE']=y1#"Strike"].iat[0]
                        group.loc[idi,'OPT_Close_PE']=y2#x["OPT_Close"].iat[0]
                        group.loc[idi,'Ticker_PE']=y3#x["Ticker"].iat[0]
                        group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                    if row['Marker']=='L':
                        if group.loc[idi,'Strike_PE']>=group.loc[idi,'EQ_Close_CE']:
                            # print(y1,group.loc[idi,'EQ_Close_CE'],date)
                            group.loc[idi,'OPT_Close_PE']=-group.loc[idi,'EQ_Close_CE']+group.loc[idi,'Strike_PE']
                        else:
                            group.loc[idi,'OPT_Close_PE']=0.05
                        if group.loc[idi,'Strike_CE']<=group.loc[idi,'EQ_Close_CE']:
                            group.loc[idi,'OPT_Close_CE']=+group.loc[idi,'EQ_Close_CE']-group.loc[idi,'Strike_CE']
                        else:
                            group.loc[idi,'OPT_Close_CE']=0.05
                # if temp==1:
                #     if row['Marker3']=='F':

                # print(group)

    return group
        
    

direc=r'D:\QODE\calender\Calendar\data\with_contract_vol_filter/'
exp_df = pd.read_csv(r'D:\QODE\straddles\Straddles/WeeklyExpiry.csv',parse_dates = ["Weekly_Expiry_Date","date"],dayfirst =True,usecols = ["Weekly_Expiry_Date","date"])
delta_list1=['50']
for delta in tqdm(delta_list):
    print(delta)
    idx = pd.read_csv(direc + "/BANKNIFTY_Weekly-I.csv",usecols = ["Date","Ticker","Option_Type","Strike",
                                                                        "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike",'At_The_Money'],parse_dates = ["Date"])

    idx2 = pd.read_csv( direc + "/BANKNIFTY_Weekly-II.csv",usecols = ["Date","Ticker","Option_Type","Strike",
                                                                        "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"],parse_dates = ["Date"])
    idx3 = pd.read_csv( direc + "/BANKNIFTY_Weekly-III.csv",usecols = ["Date","Ticker","Option_Type","Strike",
                                                                        "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"],parse_dates = ["Date"])
    idx=idx[idx['Date']<=pd.to_datetime('22-07-2021')].reset_index(drop=True)
    idx2=idx2[idx2['Date']<=pd.to_datetime('22-07-2021')].reset_index(drop=True)
    idx3=idx3[idx3['Date']<=pd.to_datetime('22-07-2021')].reset_index(drop=True)
    final2=pd.DataFrame(columns=['Delta','N','XIRR','DD','CAR/MDD'])
    
    for nnn in tqdm(N1):
            for t in trigger:
                nnn1=nnn

                output_path=r'D:\QODE\straddles\Straddles\Premium_Based_Adj/new'+'/'+delta+'_delta'+'/'+str(nnn)+'/'+str(t)
                print(output_path)
                # print(output_path)
                if not os.path.exists(output_path):
                    os.makedirs(output_path)

                # xxx


                idx_ce = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
                idx_pe = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")

                final=idx_ce.merge(idx_pe,left_on='Date_CE',right_on='Date_PE')
                final.drop(['At_The_Money_CE','At_The_Money_PE'], axis=1,inplace=True)

                final = pd.merge(final, exp_df, left_on = 'Date_CE', right_on='date',how = 'left')

                final = final.T.drop_duplicates().T  #Drop Duplicates Columns
                final.rename(columns = {'Date_CE':'Date'}, inplace = True)

                final.to_csv(output_path+"/check-1data-w3.csv")


                final=pd.read_csv(output_path+"/check-1data-w3.csv",parse_dates=['Date','Weekly_Expiry_Date'],dayfirst=True)

                df=pd.read_csv(output_path+"/check-1data-w3.csv")
                status=1
                if nnn>0:
                    nnn1=nnn
                    nnn1=nnn1-1
                # print(2,nnn1,nnn)
                for idi,row in df.iterrows():
                # df.loc[idi,'status']=status
                    if nnn!=4:
                        if idi==0:
                            if nnn!=0:
                                df.loc[idi:idi+nnn1-1,'Status']=0
                                df.loc[idi+nnn1,"Marker"]='F'
                                df.loc[idi+nnn1,'status']=1
                            else:
                                df.loc[idi,"Marker"]='F'
                                df.loc[idi,'status']=1
                        elif (df.loc[idi-1,'Date'] == df.loc[idi-1,'Weekly_Expiry_Date']):
                            # df.loc[idi,"Marker"]='M'
                            if nnn!=0:
                                # print(idi)
                                df.loc[idi:idi+nnn1-1,"Status"]=0
                            df.loc[idi+nnn1,"Marker"]='F'
                            df.loc[idi+nnn1,"status"]=1
                            # df.loc[idi+nnn-1,"status"]=1
                            df.loc[idi+nnn1,"Marker2"]='F'

                        elif (df.loc[idi,'Date'] == df.loc[idi,'Weekly_Expiry_Date']):
                            df.loc[idi,"Marker"]='L'
                            df.loc[idi,'status']=1
                    elif nnn==4:
                        if idi==0:
                                df.loc[idi+nnn1,"Marker"]='F'
                                df.loc[idi+nnn1,'status']=1
                        elif (df.loc[idi,'Date'] == df.loc[idi,'Weekly_Expiry_Date']):
                            df.loc[idi-1,"Marker"]='F'
                            df.loc[idi,"Marker"]='L'
                            df.loc[idi-1:idi,'status']=1
                        if df.loc[idi,'status']!=1:
                            df.loc[idi,'Status']=0

                df.reset_index(drop=True,inplace=True)    
                df.to_csv(output_path+'/dte_test3-3-3-1.csv')
                
                # print(3,nnn1)
                for idi,row in df.iterrows():
                    if nnn1!=0 and nnn!=4:
                        if df.loc[idi,'Status']==0:
                            df.drop(idi, axis=0, inplace=True)
                    if nnn==4:
                        if df.loc[idi,'status']!=1:

                            df.drop(idi, axis=0, inplace=True)

                df.to_csv(output_path+'/dte_test3-3-3.csv')
                # print(4,nnn1)
                from tqdm.notebook import tqdm
                final=pd.DataFrame()
                dfg=df.groupby(['Weekly_Expiry_Date'])
                temp=0
                # print(1,output_path)
                for name,group in tqdm(dfg):
                    
                    if len(group)==1:
                        continue

                    if temp==0:
                        strike_ce=group["Strike_CE"].iat[0]
                        ce_optclose=group["OPT_Close_CE"].iat[0]
                        date=group["Date"].iat[0]
                        strike_pe=group["Strike_PE"].iat[0]

                    if temp==1:
                        strike_ce=strike_new_ce
                        ce_optclose=ce_close_new#group["OPT_Close_short"].iat[0]
                        date=group["Date"].iat[0]
                        strike_pe=strike_new_pe
                        pe_optclose=pe_close_new
                        # print(strike_short,short_close_new)0
                        # xxx

                    strike_ce_w2=strike_ce
                    strike_pe_w2=strike_pe


                    x1=x2=x3=y1=y2=y3=0
                    temp1=0
                    for idi,row in group.iterrows():
                        # if temp==0:
                            date=group.loc[idi,'Date']
                            x_ce=idx[(idx['Date']==date) & (idx["Option_Type"] == "CE") & (idx["Strike"]==strike_ce)]
                            x_pe=idx[(idx['Date']==date) & (idx["Option_Type"] == "PE") & (idx["Strike"]==strike_pe)]
                            y_ce=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "CE") & (idx2["Strike"]==strike_ce_w2)]
                            y_pe=idx2[(idx2['Date']==date) & (idx2["Option_Type"] == "PE") & (idx2["Strike"]==strike_pe_w2)]
                            # z=idx3[(idx3['Date']==date) & (idx3["Option_Type"] == "CE") & (idx3["Strike"]==strike_w3)]

                            if len(x_ce):
                                group.loc[idi,'Strike_CE']=x1=x_ce["Strike"].iat[0]
                                group.loc[idi,'OPT_Close_CE']=x2=x_ce["OPT_Close"].iat[0]
                                group.loc[idi,'Ticker_CE']=x3=x_ce["Ticker"].iat[0]
                                group.loc[idi,'IV_CE']=x_ce["IV"].iat[0]
                            if len(x_ce)==0:
                                group.loc[idi,'Strike_CE']=x1#x["Strike"].iat[0]
                                group.loc[idi,'OPT_Close_CE']=x2#x["OPT_Close"].iat[0]
                                group.loc[idi,'Ticker_CE']=x3#x["Ticker"].iat[0]
                                group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]
                                
                            if len(x_pe):
                                group.loc[idi,'Strike_PE']=y1=x_pe["Strike"].iat[0]
                                group.loc[idi,'OPT_Close_PE']=y2=x_pe["OPT_Close"].iat[0]
                                group.loc[idi,'Ticker_PE']=y3=x_pe["Ticker"].iat[0]
                                group.loc[idi,'IV_PE']=x_pe["IV"].iat[0]
                            if len(x_pe)==0:
                                group.loc[idi,'Strike_PE']=y1#"Strike"].iat[0]
                                group.loc[idi,'OPT_Close_PE']=y2#x["OPT_Close"].iat[0]
                                group.loc[idi,'Ticker_PE']=y3#x["Ticker"].iat[0]
                                group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                            if row['Marker']=='L':
                                if group.loc[idi,'Strike_PE']>=group.loc[idi,'EQ_Close_CE']:
                                    # print(y1,group.loc[idi,'EQ_Close_CE'],date)
                                    group.loc[idi,'OPT_Close_PE']=-group.loc[idi,'EQ_Close_CE']+group.loc[idi,'Strike_PE']
                                else:
                                    group.loc[idi,'OPT_Close_PE']=0.05
                                if group.loc[idi,'Strike_CE']<=group.loc[idi,'EQ_Close_CE']:
                                    group.loc[idi,'OPT_Close_CE']=+group.loc[idi,'EQ_Close_CE']-group.loc[idi,'Strike_CE']
                                else:
                                    group.loc[idi,'OPT_Close_CE']=0.05
                            if nnn1==0 and nnn==0:
                                if row['Marker']=='L':

                                    yy_ce=idx2[(idx2['Date']==row['Date']) & (idx2["Option_Type"] == "CE")].reset_index(drop=True)
                                    strike_new_ce=yy_ce["Delta_"+delta+"_Strike"].iat[0]
                                    yy_ce=yy_ce[(yy_ce['Date']==date) & (yy_ce["Option_Type"] == "CE") & (yy_ce["Strike"]==strike_new_ce)]

                                    yy_pe=idx2[(idx2['Date']==row['Date']) & (idx2["Option_Type"] == "PE")].reset_index(drop=True)
                                    strike_new_pe=yy_pe["Delta_"+delta+"_Strike"].iat[0]
                                    yy_pe=yy_pe[(yy_pe['Date']==date) & (yy_pe["Option_Type"] == "PE") & (yy_pe["Strike"]==strike_new_pe)]

                                    group.loc[idi,'Strike_w2_ce']=yy_ce["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_w2_ce']=x2=yy_ce["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_w2_ce']=yy_ce["Ticker"].iat[0]
                                    group.loc[idi,'IV_w2_ce']=yy_ce["IV"].iat[0]    # xxx
                                    group.loc[idi,'Strike_w2_pe']=yy_pe["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_w2_pe']=y2=yy_pe["OPT_Close"].iat[0]

                                    group.loc[idi,'Ticker_w2_pe']=yy_pe["Ticker"].iat[0]
                                    group.loc[idi,'IV_w2_pe']=yy_pe["IV"].iat[0] 
                                    temp=1
                                    temp11=1
                                    ce_close_new=group.loc[idi,'OPT_Close_w2_ce']
                                    pe_close_new=group.loc[idi,'OPT_Close_w2_pe']

                                    if temp==0:
                                        temp=1
                                        continue
                                        
                            if row['Marker']=='F':
                                group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']
                                group.loc[idi, 'Upper_Breakeven'] = group.loc[idi, 'Strike_CE'] + (group.loc[idi, 'Premium'])
                                group.loc[idi, 'Lower_Breakeven'] = group.loc[idi,'Strike_PE'] - (group.loc[idi, 'Premium'])
                                group.loc[idi, 'Range']=group.loc[idi, 'Upper_Breakeven']-group.loc[idi, 'Lower_Breakeven']
                                
                                group.loc[idi, 'stop_trigger_CE'] = group.loc[idi, 'Strike_CE'] + t*(group.loc[idi, 'Premium'])/100
                                group.loc[idi, 'stop_trigger_PE'] = group.loc[idi,'Strike_PE'] - t*(group.loc[idi, 'Premium'])/100
                                group.loc[idi+1:,'stop_trigger_CE'] = np.nan
                                group.loc[idi+1:,'stop_trigger_PE'] = np.nan
                                group['stop_trigger_CE'].ffill(inplace=True)
                                group['stop_trigger_PE'].ffill(inplace=True)
                                    
                                
                    temp1=0
                    temp_a=0
                    x1=x2=x3=y1=y2=y3=0
                    temp_b=0
                    if nnn!=4:
                        for idi,row in group.iterrows():
                            group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']

                            # if nnn1!=0:
                            if nnn==0 and row['Marker']=='L':
                                ce_w2=group.loc[idi,'OPT_Close_w2_ce']
                                pe_w2=group.loc[idi,'OPT_Close_w2_pe']
                            if row['Marker']=='F' and idi!=0:
                                if nnn==0:
                                    start_premium=ce_w2+pe_w2
                                else:
                                    start_premium=group.loc[idi,'Premium']
                            if idi==0:
                                start_premium=group.loc[idi,'Premium']
                                
                            group.loc[idi,'%_Premium_collected']=(start_premium-group.loc[idi,'Premium'])/start_premium
                            
                            
                            if temp_a==1:
                                curr_date=row['Date']
                                x_ce=idx[(idx['Date']==curr_date) & (idx["Option_Type"] == "CE") & (idx["Strike"]==a1)]
                                x_pe=idx[(idx['Date']==curr_date) & (idx["Option_Type"] == "PE") & (idx["Strike"]==b1)]
                                # display(x_ce)
                                # display(x_pe)
                                if len(x_ce):
                                    group.loc[idi,'Strike_CE']=x1=x_ce["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_CE']=x2=x_ce["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_CE']=x3=x_ce["Ticker"].iat[0]
                                    group.loc[idi,'IV_CE']=x_ce["IV"].iat[0]
                                if len(x_ce)==0:
                                    group.loc[idi,'Strike_CE']=x1#x["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_CE']=x2#x["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_CE']=x3#x["Ticker"].iat[0]
                                    group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]

                                if len(x_pe):
                                    group.loc[idi,'Strike_PE']=y1=x_pe["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_PE']=y2=x_pe["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_PE']=y3=x_pe["Ticker"].iat[0]
                                    group.loc[idi,'IV_PE']=x_pe["IV"].iat[0]
                                if len(x_pe)==0:
                                    group.loc[idi,'Strike_PE']=y1#"Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_PE']=y2#x["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_PE']=y3#x["Ticker"].iat[0]
                                    group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                                # group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']  
                                
                                if row['Marker']=='L':
                                    if group.loc[idi,'Strike_PE']>=group.loc[idi,'EQ_Close_CE']:
                                        # print(y1,group.loc[idi,'EQ_Close_CE'],date)
                                        group.loc[idi,'OPT_Close_PE']=-group.loc[idi,'EQ_Close_CE']+group.loc[idi,'Strike_PE']
                                    else:
                                        group.loc[idi,'OPT_Close_PE']=0.05
                                    if group.loc[idi,'Strike_CE']<=group.loc[idi,'EQ_Close_CE']:
                                        group.loc[idi,'OPT_Close_CE']=+group.loc[idi,'EQ_Close_CE']-group.loc[idi,'Strike_CE']
                                    else:
                                        group.loc[idi,'OPT_Close_CE']=0.05
                                group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']
                                group.loc[idi, 'Upper_Breakeven'] = group.loc[idi, 'Strike_CE'] + (group.loc[idi, 'Premium'])
                                group.loc[idi, 'Lower_Breakeven'] = group.loc[idi,'Strike_PE'] - (group.loc[idi, 'Premium'])
                                group.loc[idi, 'Range']=group.loc[idi, 'Upper_Breakeven']-group.loc[idi, 'Lower_Breakeven']

                            if (group.loc[idi,'stop_trigger_CE'] <= group.loc[idi,'EQ_Close_CE']) or (group.loc[idi,'stop_trigger_PE'] >= group.loc[idi,'EQ_Close_CE']):
                                
                                if row['Marker']!='L':# and group.loc[idi+1,'Marker']!='L':
                                    temp_b=0
                                    group.loc[idi,'Status1']=1
                                    shift_date= (group.loc[idi,'Date'])
                                    ce_opt=group.loc[idi,'OPT_Close_CE']
                                    pe_opt=group.loc[idi,'OPT_Close_PE']
                                    idxce=idx[(idx['Date']==shift_date) & (idx["Option_Type"] == "CE") ].reset_index(drop=True)
                                    idxpe=idx[(idx['Date']==shift_date) & (idx["Option_Type"] == "PE") ].reset_index(drop=True)
                                    ce_idx=closest_strike(idxce,(ce_opt+pe_opt)/2,'CE')
                                    pe_idx=closest_strike(idxpe,(ce_opt+pe_opt)/2,'PE')
                                    # print(shift_date,ce_idx,pe_idx)
                                    ce_new=idxce.iloc[[ce_idx]].reset_index(drop=True)#idx[(idx['Date']==shift_date) &(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "CE")].reset_index(drop=True)
                                    pe_new=idxpe.iloc[[pe_idx]].reset_index(drop=True)#idx[(idx['Date']==shift_date)& (idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "PE")].reset_index(drop=True)
                                    
                                    group.loc[idi,'new_Strike_CE']=x1=a1=ce_new["Strike"].iat[0]
                                    group.loc[idi,'new_OPT_Close_CE']=x2=a2=ce_new["OPT_Close"].iat[0]
                                    group.loc[idi,'new_Ticker_CE']=a3=x3=ce_new["Ticker"].iat[0]
                                    group.loc[idi,'new_IV_CE']=ce_new["IV"].iat[0]
                                    group.loc[idi,'new_Strike_PE']=y1=b1=pe_new["Strike"].iat[0]
                                    group.loc[idi,'new_OPT_Close_PE']=y2=b2=pe_new["OPT_Close"].iat[0]
                                    group.loc[idi,'new_Ticker_PE']=y3=b3=pe_new["Ticker"].iat[0]
                                    group.loc[idi,'new_IV_PE']=pe_new["IV"].iat[0]
                                    group.loc[idi,'Premium_new']=group.loc[idi,'new_OPT_Close_CE']+group.loc[idi,'new_OPT_Close_PE']
                                    group.loc[idi, 'Upper_Breakeven'] = group.loc[idi, 'new_Strike_CE'] + (group.loc[idi, 'Premium_new'])
                                    group.loc[idi, 'Lower_Breakeven'] = group.loc[idi,'new_Strike_PE'] - (group.loc[idi, 'Premium_new'])
                                    group.loc[idi, 'Range']=group.loc[idi, 'Upper_Breakeven']-group.loc[idi, 'Lower_Breakeven']
                                    group.loc[idi, 'stop_trigger_CE'] = group.loc[idi,'new_Strike_CE'] + t*(group.loc[idi, 'Premium_new'])/100
                                    group.loc[idi, 'stop_trigger_PE'] = group.loc[idi,'new_Strike_PE'] - t*(group.loc[idi, 'Premium_new'])/100
                                    group.loc[idi+1:,'stop_trigger_CE'] = np.nan
                                    group.loc[idi+1:,'stop_trigger_PE'] = np.nan
                                    group['stop_trigger_CE'].ffill(inplace=True)
                                    group['stop_trigger_PE'].ffill(inplace=True)
                                    temp_a=1
                                    continue
                            
                        final=final.append(group,ignore_index=True)

                    else:
                        final=final.append(group,ignore_index=True)
                
                final.to_csv(output_path+'/t3-w3-3-2.csv')
                
                # print(2,output_path)
                # print(5,nnn1)
                df=pd.read_csv(output_path+'/t3-w3-3-2.csv')

                
                df['lotsize']=1
                df['qty']=1

                df['Exposure']=df['EQ_Close_CE']*2*df['lotsize']
                df['Margin']=df['Exposure']/leverage


                initial_equity=10000
                df['units'] = (initial_equity/df['Margin']).astype(int)
                df['qty_final'] = df["qty"]*df['units']

                df.to_csv(output_path+"/unit check1=w3.csv")


                df=pd.read_csv(output_path+"/unit check1=w3.csv")
                # print(6,nnn1)
                temp=0
                if nnn==4:
                    for idi, row in df.iterrows():

                        if idi==0:
                            continue
                        
                        df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                        df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                        df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])
                        if row["Marker"] == "F":
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                else:       
                    for idi, row in df.iterrows():
                        
                        if idi==0:
                            continue

                        if row['Marker2']=='L':# and row['Marker']!='L':
                            df.loc[idi,'CE_Points']=0 #-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                            df.loc[idi,'PE_Points']=0 #-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                            df.loc[idi,'Points']=0 #(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])

                        else:
                            if nnn>0:
                                if row["Marker"] == "F":
                                    df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                                elif df.loc[idi-1,'Status1']==1:
                                    df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "new_OPT_Close_PE"]
                                    df.loc[idi,'Points']=df.loc[idi,'CE_Points']+df.loc[idi,'PE_Points']
                                
                                else:
                                    df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                                    df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])
                                
                            else:
                                if temp==0:
                                    df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                                    df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])
                                if df.loc[idi-1,'Status1']==1:
                                    df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "new_OPT_Close_PE"]
                                    df.loc[idi,'Points']=df.loc[idi,'CE_Points']+df.loc[idi,'PE_Points']
                                
                                if row["Marker"] == "L" and temp==0:
                                    temp=1
                                    continue
                                if temp==1:

                                    if row["Marker"] == "F" :
                                        df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_w2_ce"]
                                        df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_w2_pe"]
                                        df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_w2_pe"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_w2_ce"])
                                    if df.loc[idi-1,'Status1']==1:
                                        df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_CE"]
                                        df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "new_OPT_Close_PE"]
                                        df.loc[idi,'Points']=df.loc[idi,'CE_Points']+df.loc[idi,'PE_Points']
                                    else:
                                        df.loc[idi,'CE_Points']=-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                                        df.loc[idi,'PE_Points']=-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                                        df.loc[idi,'Points']=(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])

                df['Total_points']=df['Points'].cumsum()
                # print(7,nnn1)
                df.to_csv(output_path+'/pl_points_'+delta+'.csv')

                df=pd.read_csv(output_path+'/pl_points_'+delta+'.csv')
                temp=0
                df['Total_points']=df['Points'].cumsum()
                df['eq_curve']=df['Points']
                df.loc[0,'eq_curve']=25000
                df['eq_curve']=df['eq_curve'].cumsum()
                df.to_csv(output_path+'/eq_curve_'+delta+'_'+str(t)+'.csv')
                print('Done')
                # print(7,nnn1)
                # xxx
                
                
                
                